In [1]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [2]:
from openai import OpenAI
import time

# Initialize the OpenAI client
client = OpenAI()

# Setup a Simple PDF Search Summary Agent

1. Ingest files (pdfs)
2. Add functionalities: 
   1. File search over the pdfs
   2. Ability to write and save files as .md so we can read them locally
3. Test it across some examples 

## 1. Ingest files

In [4]:
def create_vector_store():
    vector_store = client.beta.vector_stores.create(
        name="PDFs",
    )
    return vector_store

vector_store = create_vector_store()

vector_store_id = vector_store.id

In [5]:
!wget -O "paper.pdf" "https://arxiv.org/pdf/1706.03762"

--2025-01-13 17:27:23--  https://arxiv.org/pdf/1706.03762
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.195.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘paper.pdf’

paper.pdf           100%[===================>]   2.11M  --.-KB/s    in 0.03s   

2025-01-13 17:27:24 (73.2 MB/s) - ‘paper.pdf’ saved [2215244/2215244]



In [6]:
!wget -O "ai-agents-paper.pdf" "https://arxiv.org/pdf/2308.11432"

--2025-01-13 17:28:02--  https://arxiv.org/pdf/2308.11432
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.195.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5789022 (5.5M) [application/pdf]
Saving to: ‘ai-agents-paper.pdf’

ai-agents-paper.pdf 100%[===================>]   5.52M  --.-KB/s    in 0.06s   

2025-01-13 17:28:02 (96.4 MB/s) - ‘ai-agents-paper.pdf’ saved [5789022/5789022]



In [7]:
!mkdir -p "pdfs"
!mv "paper.pdf" "pdfs/paper.pdf"
!mv "ai-agents-paper.pdf" "pdfs/ai-agents-paper.pdf"

In [8]:
import glob
file_paths = glob.glob("pdfs/*.pdf")

file_paths


['pdfs/ai-agents-paper.pdf', 'pdfs/paper.pdf']

In [9]:
def add_files_to_vector_store(file_paths, vector_store_id):

    """
    Upload files to the vector store and wait for processing to complete
    """
    # Prepare the files for upload
    file_streams = [open(path, "rb") for path in file_paths]
    
    try:
        # Upload files and wait for processing
        file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
            vector_store_id=vector_store_id,
            files=file_streams
        )
        
        print(f"Upload status: {file_batch.status}")
        print(f"File counts: {file_batch.file_counts}")
        return file_batch
        
    finally:
        # Clean up file streams
        for stream in file_streams:
            stream.close()

add_files_to_vector_store(file_paths, vector_store_id)

Upload status: completed
File counts: FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


VectorStoreFileBatch(id='vsfb_1d02d274bd4d4e1f8baceaf768adabd2', created_at=1736789336, file_counts=FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2), object='vector_store.file_batch', status='completed', vector_store_id='vs_2wyyYIKMHrZMYZnlGtJC21Gm')

In [12]:
assistant = client.beta.assistants.create(
    name="PDF Search Assistant",
    instructions="You are a helpful assistant that can search and summarize the available PDFs and answer questions about them.",
    tools=[{"type": "file_search"},
           {"type": "code_interpreter"}],
    model="gpt-4o"
)

assistant.id

'asst_yDk3m3FqUuj799coN6CPdj8F'

In [13]:
# Create vector store
vector_store = create_vector_store()

# Update assistant with vector store
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tools=[{"type": "file_search"},
           {"type": "code_interpreter"}],
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store.id]
        },
    }
)

In [26]:
thread = client.beta.threads.create()

messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Summarize the documents available."
)

messages

Message(id='msg_Y5c9XYflrV6Dz1G0PVbNDEvf', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Summarize the documents available.'), type='text')], created_at=1736790359, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_AFp803nslOt4FENRD1bgFWYA')

In [27]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

print(run)

if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
    )
    print(messages)
else:
    print(run.status)

Run(id='run_GbdFeQcpnWb2LSoHDyIskQwB', assistant_id='asst_yDk3m3FqUuj799coN6CPdj8F', cancelled_at=None, completed_at=1736790365, created_at=1736790361, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant that can search and summarize the available PDFs and answer questions about them.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1736790363, status='completed', thread_id='thread_AFp803nslOt4FENRD1bgFWYA', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21'))), CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=68, prompt_tokens=1911, total_toke

In [28]:
messages.data[0].content[0].text.value

"It seems I wasn't able to retrieve a summary of your documents automatically. Please let me know if there's a specific part you'd like summarized or if you can provide some context about the content. I can then search through the documents more effectively to provide a detailed summary."